In [ ]:
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import geojson

In [ ]:
minlon=48
maxlon=58.5
minlat=25
maxlat=34.2

In [ ]:
#For now I'm hardwiring in 100 years, if you change the timespan, change the variable below, which is used later
numyears=100

parameters={
    "starttime":"1913-01-01",
    "endtime":"2013-12-31",
    "minmagnitude":4,
    "minlatitude":minlat,
    "maxlatitude":maxlat,
    "minlongitude":minlon,
    "maxlongitude":maxlon,
    "limit":20000,
    "format":"geojson"
}
response=requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query",params=parameters)
output=response.json()
numquakes = len(output['features'])
print(numquakes) 
#note that if you hit 20,000 you've maxed out and need to look at a smaller area or time span.  
#Otherwise, you will be THINKING you have 100 years of earthquakes and in reality you will 
#have 75 or something like that and the "divide N by timespan" step below will be wrong.


In [ ]:
lon=np.empty(numquakes,)
lat=np.empty(numquakes,)
z=np.empty(numquakes,)
mag=np.empty(numquakes,)
for i in range(numquakes):
    lon[i] = output['features'][i]['geometry']['coordinates'][0]
    lat[i] = output['features'][i]['geometry']['coordinates'][1]
    z[i]   = output['features'][i]['geometry']['coordinates'][2]   
    mag[i] = output['features'][i]['properties']['mag']
    

In [ ]:
fig=plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([minlon,maxlon,minlat,maxlat])
ax.coastlines()
ax.stock_img()

im=ax.scatter(lon,lat,s=4,c=z,cmap='rainbow')

gl = ax.gridlines(draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.ylabel_style = {'size': 15, 'color': 'gray'}
gl.xlabel_style = {'color': 'red', 'weight': 'bold'}
fig.colorbar(im,ax=ax,label='Depth (km)',orientation='horizontal')
plt.show()


In [ ]:
deltamag=0.01
bins=np.arange(4,10,deltamag)
numbins=len(bins)
N=np.empty(numbins,)
for i in range(numbins):
    goodmag = np.where((mag >= bins[i]))
    N[i]=np.shape(goodmag)[1]
 
N[N == 0] = 'nan' # or use np.nan
goodid = np.where(np.isfinite(N))
N = N/numyears


bestline = np.polyfit(bins[goodid], np.log10(N[goodid]), 1)
print('b   = ' + str(-bestline[0]) + ', a = ' + str(bestline[1]))
line = bestline[0]*bins + bestline[1]
print('a/b = '+str(-bestline[1]/bestline[0]))


In [ ]:
fig=plt.figure(figsize=(10,10))

plt.plot(bins,np.power(10,line),'r-')
plt.semilogy(bins,N,'*')
plt.grid()
plt.ylabel('N, events/year with magnitude >M')
plt.xlabel('Magnitude, M')
plt.legend(('G-R fit','all data'))

plt.show()

In [ ]:
#Now cull out points in this plot below a guess of the "magnitude completeness threshold"
mineq=4.3
maxeq=8

goodid = np.where(np.isfinite(N) & (bins > mineq) & (bins<maxeq))

bestline = np.polyfit(bins[goodid], np.log10(N[goodid]), 1)
print('b   = ' + str(-bestline[0]) + ', a = ' + str(bestline[1]))
line = bestline[0]*bins + bestline[1]
print('a/b = '+str(-bestline[1]/bestline[0]))

In [ ]:
fig=plt.figure(figsize=(10,10))

plt.plot(bins,np.power(10,line),'r-')
plt.semilogy(bins,N,'*')
plt.semilogy(bins[goodid],N[goodid],'ro')
plt.grid()
plt.ylabel('N, events/year with magnitude >M')
plt.xlabel('Magnitude, M')
plt.legend(('G-R fit','all data','data after culling'))
plt.show()

In [ ]:
##Total seismic moment/year

#this equation is wrong
m0 = np.log10(1.5*mag+np.power(10,9.1))
m0peryear = np.sum(m0)/numyears

#fix this equation too!!!
mwperyear = m0peryear-np.sin(np.sum(mag))
print('Average moment realease in Newton-meters per year: '+str(m0peryear))
print('Average Mw per year'+str(mwperyear))